In [ ]:
# %% Cell 1: Import libraries
import os
import numpy as np
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio.mask



print("✅ Libraries imported successfully")



In [ ]:
# Load the Finland shapefile (Update the path if needed)
finland_shp = gpd.read_file("/Data/fi_shp/fi-MultiPolygon.shp")

# Ensure it's in the same CRS as your temperature raster
finland_shp = finland_shp.to_crs("EPSG:4326")

# Display the shapefile to verify
finland_shp.plot()
plt.title("Finland Boundaries")
plt.show()

In [ ]:
def clip_raster_to_shapefile(raster_path, shapefile, output_path):
    with rasterio.open(raster_path) as src:
        # Convert the shapefile geometry into rasterio format
        shapes = [shapefile.geometry.unary_union.__geo_interface__]

        # Clip the raster using the mask function
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta.copy()

        # Update metadata to match the clipped image
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # Save the clipped raster
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

        print(f"✅ Clipped and saved: {output_path}")

# Example usage:
clip_raster_to_shapefile("/Data/temperature.tif", finland_shp, "/Data/clipped_temperature.tif")

In [ ]:
# %% Cell 4: Configuration
# Input directory containing your GeoTIFF files
geotiff_file = "/Data/clipped_temperature.tif"  # Update this path
finland_shapefile = "/Data/fi_shp/fi-MultiPolygon.shp"  # Update this path
output_gif = "temperature_animation.gif"
temp_image_dir = "temp_images"


In [ ]:
# Create directories if they don't exist
os.makedirs(temp_image_dir, exist_ok=True)

print("⚙️ Configuration set:")
print(f"- GeoTIFF file: {geotiff_file}")
print(f"- Shapefile: {finland_shapefile}")
print(f"- Output GIF: {output_gif}")

In [ ]:
# %% Cell 6: Load Shapefile and Raster Data
print("🔍 Loading Finland shapefile...")

try:
    # Load the shapefile using geopandas
    finland_gdf = gpd.read_file(finland_shapefile)
    print("✅ Shapefile loaded successfully!")
    print("Here's a preview of the shapefile data:")
    print(finland_gdf.head())
except Exception as e:
    print("❌ Error loading shapefile:")
    print(e)
    raise

print("\n🔍 Loading GeoTIFF temperature data...")

try:
    # Open the GeoTIFF file using rasterio
    raster = rasterio.open(geotiff_file)
    print("✅ GeoTIFF loaded successfully!")
    print("Raster information:")
    print(" - Coordinate Reference System (CRS):", raster.crs)
    print(" - Raster bounds:", raster.bounds)
    print(" - Number of bands (assumed time steps):", raster.count)
except Exception as e:
    print("❌ Error loading GeoTIFF:")
    print(e)
    raise


In [ ]:
# %% Cell 7: Process Temperature Data and Save Frames with Map Enhancements
import pandas as pd  # for creating date range
from matplotlib.colors import Normalize
import math

print("📅 Creating date range for temperature bands...")
# Create a date range with monthly frequency, assuming each band is one month.
dates = pd.date_range(start="2004-01-01", periods=raster.count, freq="MS")  # 'MS' means Month Start
print(f"✅ Date range created with {len(dates)} timestamps.")

print("\n🔄 Reprojecting shapefile to WGS84 (EPSG:4326) for overlay...")
try:
    # Convert the shapefile to WGS84 so that it aligns with our raster (which uses geographic coordinates)
    finland_gdf_wgs84 = finland_gdf.to_crs(epsg=4326)
    print("✅ Shapefile reprojected successfully!")
except Exception as e:
    print("❌ Error reprojecting shapefile:")
    print(e)
    raise

# Define normalization for temperature display (in Celsius)
vmin, vmax = -30, 30

# Define a function to add a scale bar to the map
def add_scalebar(ax, length, location=(0.1, 0.1), linewidth=3):
    """
    Adds a scale bar to the axes.
    
    Parameters:
        ax (matplotlib.axes.Axes): The axis to add the scale bar to.
        length (float): The length of the scale bar in kilometers.
        location (tuple): Fractional location (x, y) in axis coordinates.
        linewidth (int): Line width of the scale bar.
    """
    # Get the current axis limits in data coordinates
    x0, x1 = ax.get_xlim()
    y0, y1 = ax.get_ylim()
    # Determine the starting point of the scale bar in data coordinates
    sb_x = x0 + (x1 - x0) * location[0]
    sb_y = y0 + (y1 - y0) * location[1]
    # Convert the desired length (in km) to degrees.
    # Approximation: 1 degree latitude ~ 111 km; adjust longitude using the cosine of the mid-latitude.
    mid_lat = (y0 + y1) / 2.0
    km_per_deg = 111 * math.cos(math.radians(mid_lat))
    deg_length = length / km_per_deg
    # Draw the scale bar as a horizontal line
    ax.hlines(sb_y, sb_x, sb_x + deg_length, colors='k', linewidth=linewidth)
    # Label the scale bar in the center
    ax.text(sb_x + deg_length / 2, sb_y - (y1 - y0) * 0.02, f'{length} km',
            ha='center', va='top', fontsize=10)

print("\n🖼️ Processing each band to create frames for the animation...")

# Loop over each band in the raster
for band in range(1, raster.count + 1):
    print(f"\nProcessing band {band}/{raster.count}...")
    
    try:
        # Read the band data from the raster. Ensure the data is float for processing.
        temp_data = raster.read(band).astype(float)
    except Exception as e:
        print(f"❌ Error reading band {band}:")
        print(e)
        continue
    
    # Temperature data is assumed to be already in Celsius
    temp_data_celsius = temp_data

    # Debug: print the min and max temperature for this band
    band_min = np.nanmin(temp_data_celsius)
    band_max = np.nanmax(temp_data_celsius)
    print(f"   Temperature range for band {band}: {band_min:.2f}°C to {band_max:.2f}°C")
    
    # Create a figure and axis for plotting
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Plot the temperature data using imshow
    img = ax.imshow(
        temp_data_celsius,
        cmap='coolwarm',
        extent=(raster.bounds.left, raster.bounds.right, raster.bounds.bottom, raster.bounds.top),
        origin='upper',
        norm=Normalize(vmin=vmin, vmax=vmax)
    )
    
    # Overlay the Finland shapefile boundaries
    finland_gdf_wgs84.boundary.plot(ax=ax, color='black', linewidth=1)
    
    # Set axis labels to display coordinates and add a grid
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.grid(True, linestyle='--', alpha=0.5)
    
    # Add a title for the specific time step
    ax.set_title(f"Temperature Pattern - {dates[band-1].strftime('%Y-%m')}")
    
    # Add a main title for the map (using a suptitle)
    fig.suptitle("Finland Temperature Map", fontsize=16)
    
    # Add a colorbar to display the temperature scale
    fig.colorbar(img, ax=ax, orientation='vertical', label='Temperature (°C)')
    
    # Add a scale bar (e.g., 100 km) in the bottom left corner
    add_scalebar(ax, 100, location=(0.7, 0.1))
    
    # Add a north arrow in the top left corner
    ax.annotate('N', xy=(0.05, 0.95), xytext=(0.05, 0.85),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                ha='center', va='center', fontsize=12, xycoords=ax.transAxes)
    
    # Save the current frame as a PNG image
    frame_filename = os.path.join(temp_image_dir, f"frame_{band:03d}.png")
    try:
        plt.savefig(frame_filename, bbox_inches='tight')
        print(f"✅ Saved frame: {frame_filename}")
    except Exception as e:
        print(f"❌ Error saving frame {band}:")
        print(e)
    
    # Close the figure to free memory
    plt.close(fig)

print("\n🎉 All frames processed and saved successfully!")


In [ ]:
# %% Cell 8: Create GIF Animation from Frames with Updated Resampling
print("🎞️ Starting GIF creation process...")

# Use imageio.v2 to avoid deprecation warnings
import imageio.v2 as imageio
from PIL import Image

# List to hold file paths of the frame images
frame_files = []

# Get all PNG files from the temporary image directory and sort them
try:
    for filename in sorted(os.listdir(temp_image_dir)):
        if filename.endswith(".png"):
            frame_path = os.path.join(temp_image_dir, filename)
            frame_files.append(frame_path)
    print(f"✅ Found {len(frame_files)} frame images.")
except Exception as e:
    print("❌ Error accessing frame images:")
    print(e)
    raise

# Check if there are any frames to process
if not frame_files:
    raise FileNotFoundError("No frame images were found in the temporary directory.")

# Read the first image to determine the target shape (width, height)
try:
    with Image.open(frame_files[0]) as img:
        target_size = img.size  # (width, height)
    print(f"🎯 Target size for all frames: {target_size}")
except Exception as e:
    print("❌ Error reading the first frame to determine target size:")
    print(e)
    raise

# Read all frames, resizing them if necessary, and store them in a list
frames = []
for frame_file in frame_files:
    try:
        # Open the image using Pillow
        with Image.open(frame_file) as img:
            # Resize if the image size doesn't match the target
            if img.size != target_size:
                img = img.resize(target_size, resample=Image.LANCZOS)
            # Convert the image to a numpy array
            frame_array = np.array(img)
            frames.append(frame_array)
    except Exception as e:
        print(f"❌ Error processing frame {frame_file}:")
        print(e)

print("✅ All frames loaded and resized successfully. Creating GIF...")

# Create the GIF using imageio
try:
    # 'duration' controls the time (in seconds) each frame is shown (0.2 seconds here)
    imageio.mimsave(output_gif, frames, duration=0.4)
    print(f"🎉 GIF created successfully and saved as '{output_gif}'!")
except Exception as e:
    print("❌ Error creating GIF:")
    print(e)
    raise
